In [1]:
# import list
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random

from nltk import sent_tokenize

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import multilabel_confusion_matrix

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import transformers
from transformers import AutoModel, BertTokenizerFast
from transformers import BertModel, DistilBertTokenizer, DistilBertModel
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModelForMaskedLM

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import AlbertModel

from transformers import ElectraModel, ElectraTokenizer
from transformers import XLNetTokenizer, XLNetModel

from datasets import load_dataset

## 오늘 날짜 확인
import datetime as dtime
from datetime import datetime, timedelta

## SSH 연결 
import paramiko

%matplotlib inline

In [5]:
transformers.__version__

'4.25.1'

In [ ]:
### 여기서부터는 input 처리 부분! 추후에 함수화 할 것! 모듈화! 

In [5]:
model_path = 'google/electra-base-discriminator'  # "monologg/koelectra-base-v3-discriminator"
model =   ElectraModel.from_pretrained(model_path)  # KoELECTRA-Small-v3
tokenizer =  ElectraTokenizer.from_pretrained(model_path )

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
# 저장한 모델 가중치 불러오기 
device = torch.device('cpu')   # GPU 없을시, CPU로 해야함 단점은 속도가 느림!
set_seed(42)    # Set seed for reproducibility
#model = HalfAgreeClassifier()
model = AllAgreeClassifier()

out_dir = './model' 
## saved_weights_220629.pt  5 epoch 모델 
#load weights of best model
#path = out_dir+ '/' + 'saved_weights_2303_acc86(50agree).pt'
path = out_dir+ '/' + 'saved_weights_2303_acc96(allagree).pt'
print(path)
model.load_state_dict(torch.load(path))  #, map_location=device   / strict = False 

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


./model/saved_weights_2303_acc96(allagree).pt


<All keys matched successfully>

In [7]:
# model parameter
MAX_LEN = 64
batch_size = 16

# ssh info
host  = '121.254.150.83'
port_num  = 6502
user_id = 'bascrap'
user_password = 'qwe123!@#'

# target date 
yesterday = datetime.today() - timedelta(1)
yesterday_date = yesterday.strftime("%Y%m%d")
## 오전 10 ~ 2시 사이에 업데이트? 
yesterday_date = '20230323'  # 테스트용 

# ssh 타겟 경로 
dir_path = '/home/bascrap/data/stock/investing_news/'
folder_path = yesterday_date + '/'
remote_dir_path = dir_path + folder_path 
remote_dir_path

'/home/bascrap/data/stock/investing_news/20230323/'

In [8]:
# main run 
target_list, ssh = ssh_access_day_files(host, port_num, user_id, user_password, remote_dir_path)
result_list = sentiments_score_extract(target_list, remote_dir_path, ssh, tokenizer, MAX_LEN, batch_size)


## 종목, 일짜 합산 평균 
stock_dic = stock_dic_make(result_list)
ticker_df = ticker_output(stock_dic)  # 종목 출력 
day_avg_preds = day_output(stock_dic)  #하루 출력 
day_df = yesterday, day_avg_preds

C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
## 전체 출력 및 확인용 
len(result_list), len(target_list)

(198, 198)

In [10]:
ticker_df

,symbol,sentiment score
0,AAPL,0.000000
1,ABBV,0.225275
2,ABB,1.000000
3,ACN,0.393939
4,ADBE,0.294118
...,...,...
128,WFC,-0.071429
129,WMT,0.104167
130,XL,-0.500000
131,XOM,-0.428571


In [11]:
day_df

(datetime.datetime(2023, 3, 29, 10, 38, 20, 680936), 0.09332147310540403)

In [22]:
symbol_input =  'ABB'
on_symbol = ticker_df[ticker_df['symbol'] == symbol_input]
on_symbol

,symbol,sentiment score
2,ABB,1.0


In [15]:
## 후속 작업, 라이브러리를 활용해서 일짜별 종목 주가 불러오기 : 모델의 진짜 성능 평가 
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [23]:
# 예시
stock_df = pdr.get_data_yahoo(symbol_input, "2023-03-19", "2023-03-28")
# 'Close' 컬럼을 이용하여 일별 변동률을 계산합니다.
daily_returns = stock_df['Close'].pct_change()
# 계산된 일별 변동률을 'Returns' 컬럼으로 추가합니다.
stock_df['Returns'] = daily_returns
# 결과를 출력합니다.
stock_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Returns
Date,,,,,,,
2023-03-20,32.380001,32.759998,32.320000,32.560001,31.646332,1150800,NaN
2023-03-21,33.209999,33.369999,33.099998,33.270000,32.336407,1233200,0.021806
2023-03-22,33.549999,33.779999,33.119999,33.119999,32.190617,1487700,-0.004509
2023-03-23,33.509998,33.750000,33.169998,33.380001,32.443321,2115500,0.007850
2023-03-24,32.509998,32.770000,32.169998,32.750000,31.830999,2147000,-0.018874
2023-03-27,32.160000,32.299999,32.029999,32.139999,32.139999,1235800,-0.018626


In [4]:
def sentiments_score_extract(target_list, remote_dir_path, ssh, tokenizer, MAX_LEN, batch_size):
    result_list = []

    for number in range(len(target_list)):
        symbol, title =  target_list[number].split('_')[0], target_list[number].split('_')[1]
        remote_file_path = remote_dir_path +  target_list[number]
        body = content_extract(remote_file_path)
        tokenized_text = sent_tokenize(body)
        text_list = [title] + tokenized_text
        doc_tokens = preprocessing_contents(text_list)
        preds = prediction_labels(doc_tokens, batch_size, tokenizer)
        sum_preds = sum(preds)
        avg_preds = sum(preds)/ len(preds)
        # 종목, 제목, 본문, 문장별 예측라벨, 예측 합 , 예측 평균 , 날짜
        data = [symbol, title, body, preds ,sum_preds, avg_preds ,yesterday_date ]
        result_list.append(data)
    
    ssh.close()
    return result_list

In [3]:
# ssh 이츠에이 수집한 investing.news 해외 종목 뉴스 접근 및 파일 리스트 추출
def ssh_access_day_files(host, port_num, user_id, user_password, remote_dir_path):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(hostname=host, port=port_num, username=user_id, password=user_password)

    stdin, stdout, stderr = ssh.exec_command(f'ls {remote_dir_path}')
    file_list = stdout.read()

    target_list = file_list.decode().split('\n')
    target_list = target_list[:-1]

    return target_list, ssh


# 파일 속 텍스트의 내용 추출 
def content_extract(remote_file_path):
    sftp = ssh.open_sftp()
    with sftp.open(remote_file_path) as f:
        content = f.read().decode()
        
    content_split = content.split('\n\n')

    body = ''

    for i in range(len(content_split)):
        if len(content_split[i]) > 20 :
            body = content_split[i]    
    body = body.replace('\n','')
    body = body.split('Sources:')[0].strip()
    sftp.close()        
    return body



# 모델 입력전 전처리 함수 
def preprocessing_contents(text_list):

    doc_tokens = []

    for d in range(len(text_list)):
        X_pred =  text_list[d]
        X_pred = text_preprocessing(X_pred)
        X_pred_split = X_pred.split(' ')
        X_pred = text_word_one_limit(X_pred_split)
        doc_tokens.append(X_pred)
        
    return doc_tokens

# 모델에 예측 라벨을 계산하는 함수 
def prediction_labels(doc_tokens, batch_size, tokenizer):
    pred_inputs, pred_masks = preprocessing_for_bert(doc_tokens, tokenizer)
    pred_dataset = TensorDataset(pred_inputs, pred_masks)
    pred_sampler = SequentialSampler(pred_dataset)
    pred_dataloader = DataLoader(pred_dataset, sampler=pred_sampler, batch_size= batch_size)

    probs = bert_predict(model, pred_dataloader)  
    preds = np.argmax(probs, axis = 1)

    preds = list(preds)

    results  = []

    for p in range(len(preds)):
        p_plus = preds[p]
        p_real = p_plus - 1
        results.append(p_real)
        
    return results



# output 을 위한 딕셔너리 구성
def stock_dic_make(result_list):
    stock_dic = {}
    for r in range(len(result_list)):
        in_data = result_list[r]
        symbol, avg_preds = in_data[0], in_data[5]
    
        try: 
            stock_dic[symbol].append(avg_preds)
        except: #  처음꺼라면 
            stock_dic[symbol] = []
            stock_dic[symbol].append(avg_preds)
    return stock_dic


## 종목별 감정 스코어 평균
def ticker_output(stock_dic):
    ## output 
    ticker_list = list(stock_dic.keys()) 

    ticker_data = []

    for tn in range(len(ticker_list)):
        ticker_name = ticker_list[tn]
        stock_avg_preds = sum(stock_dic[ticker_name])/len(stock_dic[ticker_name])
    
        in_ticker = [ticker_name, stock_avg_preds]
        ticker_data.append(in_ticker)

    ticker_df = pd.DataFrame(ticker_data, columns = ['symbol', 'sentiment score'])
    return ticker_df

## 일짜 전체 출력 : 여기까지 하고 잠깐 스터디!
def day_output(stock_dic):
    ticker_list = list(stock_dic.keys()) 

    total_day = []

    for t in range(len(ticker_list)):
        ticker = ticker_list[t]
        ticker_avg_preds = sum(stock_dic[ticker])/len(stock_dic[ticker])
        total_day.append(ticker_avg_preds)
    
    day_avg_preds = sum(total_day) / len(ticker_list)
    
    return day_avg_preds    

In [2]:
### 함수들 
def text_word_one_limit(t_split):

    con_text = ''
    for tn in range(len(t_split)):
        tns = t_split[tn]
        if len(tns) <= 1 :
            continue

        con_text = con_text + ' ' + tns  
    return con_text.strip()

# 전처리 함수, 추후 증권 뉴스에 맞게 일부 추가 및 수정 필요 
def text_preprocessing(text):
    cleaned_text = re.sub('[^a-zA-Z]', ' ', text)
    cleaned_text = re.sub('\n', '', cleaned_text)
    cleaned_text = re.sub('\xa0', '', cleaned_text) 
    cleaned_text = re.sub(
        '[\{\}\[\]\/?;:|\)…－〕.〔ⓘㅇ÷♠♣＜＞©◀Ⅱ·―Ⅱ＆,？☏☎™×『』《》／┌─┬┐│├ ┼┤└┴┘★〈●○[］〉±▨→↑↓∼％「」※ㆍ♥①②③④⑤⑥⑦⑧⑨△◇ ㈜ⓝ◈；：“”‘’ *~【】♡♥▽▷ⓒ▣◇□㈜◆☞■▶▲▼`!^\-_+<>@\#$%&\\\=\(\'\"]',
        ' ', cleaned_text)

    return cleaned_text


## 헤드 테일 짜르기 
def truncation_method(df, head, tail):  # 문서 리스트, head 자를 비율, tail 자를 비율 
    target_dt = []
    #head
    head_n = int(510 * head) 

    #tail
    tail_n = int(510 * tail) 
    
    print(head_n, tail_n)

    for i in range(len(df)):
        target = df['text'][i]

        target_head = target[:head_n]  # head 짜르기 
        
        if len(target) > 510  : # 510 max_len
            tail = len(target) - tail_n # 510
            target_tail = target[tail:]
            text = target_head + target_tail 
        else:
            #tail = 0
            text = target
        label = df['label'][i]
        date = df['date'][i]
        target_dt.append([label, text, date])

    return target_dt

def preprocessing_for_bert(data, tokenizer):
    input_ids = []
    attention_masks = []
    
    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            text=sent,  
            add_special_tokens=True,       
            max_length=MAX_LEN,                
            pad_to_max_length=True,              
            return_attention_mask=True,    
            truncation= True
            )
        
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks






# ======================================================================

# /model/saved_weights_2303_acc86(50agree).pt
class HalfAgreeClassifier(nn.Module):

    def __init__(self, freeze_bert=False):

        super(HalfAgreeClassifier, self).__init__()

        D_in, H, D_out = 768, 256, 3
        model_path = 'google/electra-base-discriminator'
        self.bert = ElectraModel.from_pretrained(model_path) 

        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Dropout(0.25), 
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)

        last_hidden_state_cls = outputs[0][:, 0, :]

        logits = self.classifier(last_hidden_state_cls)

        return logits
    
    
    
# ./model/saved_weights_2303_acc96(allagree).pt
class AllAgreeClassifier(nn.Module):
    def __init__(self, freeze_bert=False):
        super(AllAgreeClassifier, self).__init__()
        D_in, H, D_out = 768, 768, 3
        model_path = 'google/electra-base-discriminator'
        self.bert = ElectraModel.from_pretrained(model_path) 

        self.classifier = nn.Sequential( 
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Dropout(0.2), 
            nn.Linear(H, D_out)
        )
        
        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):

        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        last_hidden_state_cls = outputs[0][:, 0, :]

        logits = self.classifier(last_hidden_state_cls)

        return logits
    
    
# ======================================================================

    
def bert_predict(model, test_dataloader):
    model.eval()
    all_logits = []

    for batch in test_dataloader:
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    all_logits = torch.cat(all_logits, dim=0)
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs


def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    
